#Vertex Agents SDK for quering MongoDB Atlas

This colab demonstrates all how to use Vertex AI extension to query MongoDB Atlas

##Authenticate

In [1]:
from google.colab import auth
auth.authenticate_user()

##Download most recent copy of Vertex Agents SDK

In [ ]:
!gsutil cp gs://vertex_agents_private_releases/vertex_agents/google_cloud_aiplatform-1.60.dev20240710+vertex.agents-py2.py3-none-any.whl .

In [ ]:

!pip install --upgrade --force-reinstall -q google_cloud_aiplatform-1.60.dev20240710+vertex.agents-py2.py3-none-any.whl --no-warn-conflicts
!pip install numpy==1.26.4

# Restart the kernel runtime to load the private preview SDK
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

##Imports and Initialization

Make sure to initialize aiplatform with your projectID, location, and api endpoint. You need to initialize aiplatform before performing any of the other imports

In [1]:
# TODO: Replace with your project ID.
PROJECT_ID = "Your project name" # @param {type:"string"}
LOCATION = 'Your region of deployment' # @param {type:"string"}
STAGING_BUCKET =  "gs://bucket name" # @param {type:"string"}
API_ENDPOINT = 'us-central1-aiplatform.googleapis.com' # @param {type:"string"}
LLM_MODEL = "gemini-1.5-pro" # @param {type:"string"}

from google.cloud import aiplatform
from google.cloud.aiplatform.private_preview.vertex_agents.app import App, Session
from google.cloud.aiplatform.private_preview.vertex_agents.agent import Agent
from vertexai.preview.extensions import Extension

aiplatform.init(project=PROJECT_ID, location=LOCATION, api_endpoint=API_ENDPOINT)

#Apps

##Create App

There are two options for creating an app.


1.   use the App.create function, which will create a new app for you
2.   initialize a Vertex Agents App object with the resource name of a previously created App



In [2]:
app = App.create(display_name='Vertex Agents App for MongoDB', description='made via the SDK')

# To load an existing app
# app = App("projects/<project_id>/locations<location_id>/apps/<app_id>")

##List Apps

You can list all the apps in a project, which will return a list of operational Apps.

Alternatively, you can get a user-friendly mapping of App display names and resource names.

In [3]:
all_apps = App.list_apps()
all_apps[0]

app_name: projects/787220387490/locations/us-central1/apps/3874503054422179840
display_name: Vertex Agents App for MongoDB
description: made via the SDK
agents: {}

# Tools

###Extension Tools

####MongoDB Extensions

Run the 1. extensions for MongoDB notebook to generate the extension id and provide to EXTENSIONS_ID variable.

In [52]:
from google.cloud import aiplatform
from vertexai.preview import extensions

EXTENSIONS_ID="Extension ID created using extension notebook" # @param {type:"string"}

mdb_extension = extensions.Extension(EXTENSIONS_ID)

# Operations on an app

## Agents

Agents exist under an App. Agents can perform small, specific tasks utilizing Vertex Extensions and function calling. Agents require a display name, and a set of instructions.

When creating an agent, you must specify the following:


1.   Display Name
2.   Instructions - the detailed instructions the Agent should follow. This is where you instruct your agent when to call your functions / extensions and with what parameters.

You also have the option to specify the extensions and functions you will be using. Please note that if you reference an extension or function in the instructions, you ***must*** include an extension/function with the same display name in the extensions/functions lists.

##Create Agent
You can create an agent for your app by using the add_agent method. When creating an agent, you must specify all the required fields discussed above.



##Create a single Agent App

If you want to create an App and Agent in a single turn (i.e. avoiding using create_app -> add_agent) you can use `App.create_single_agent_app`. You must specify the agent display name, and then optionally specify tools for the agent, and the App's display name and description. This function returns an operable App.

**Update the </Your Cluster name here/> with your cluster name.**

In [53]:
DISPLAY_NAME = "MongoDB Agent"
INSTRUCTIONS = """
If user ask about anything from papers published use the extension MongoDb Vertex API Interpreter to make the call,
The agent makes api call to This extension makes api call to mongodb cluster '</Your Cluster name here/>', database 'sample_mflix' and collection 'movies' it also connects to database 'sample_restaurants' and collection 'restaurants' to do all crud operations only return the documents from the same.
Only respond based on the response of the tool. Do not create your own answers.
"""

app = App.create_single_agent_app(
    agent_display_name=DISPLAY_NAME,
    instructions=INSTRUCTIONS,
    extensions={
        'mongodb': mdb_extension
    }
    )

##List Agents

You can list all the agents exiting under an App. This will return operatable Agent objects.

In [ ]:
app.list_agents()

##Get Agent

You can also get a specific Agent using either the resource name or the display name

In [55]:
agent = app.get_agent('MongoDB Agent')

In [ ]:
agent

Alternatively, you can get an Agent using the Agent class

Note that the only way to create a new Agent is to create it under an App, using `app.add_agent()`. The Agents constructor can only be used to get a reference to an Agent that has already been created.

#Session

NOTE: you can remove `.content` from the end of each run call to get more detailed information about each session action

In [63]:
session = app.start_session()

## Multi-turn queries without tool calls


In [58]:
session.run("Who is the cast in the movie 'The Great Train Robbery'").content

role: "model"
parts {
  text: "The cast of the movie \'The Great Train Robbery\' includes A.C. Abadie, Gilbert M. \'Broncho Billy\' Anderson, George Barnes, and Justus D. Barnes. \n"
}

In [ ]:
session.get_history()

In [60]:
session.run("Give me a name of movie starring 'Will Smith'").content

role: "model"
parts {
  text: "A movie starring Will Smith is \'Made in America\'. \n"
}

## User query that invokes client side function


In [61]:
session.run("get me some borough for cuisine American").content

role: "model"
parts {
  text: "Some boroughs for American cuisine are Brooklyn, Queens, and Bronx. \n"
}

In [ ]:
# app.delete_session('<Session Name>')